In [1]:
import numpy as np
import pandas as pd
import time
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score


In [2]:
# Load dataset
data = load_wine()
X = data.data
y = data.target

# Normalize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

# One-hot encode labels
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

In [4]:
# TensorFlow model
tf_model = Sequential([
    Dense(32, activation='relu', input_shape=(X.shape[1],)),
    Dense(3, activation='softmax')
])

tf_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

c:\Users\Admin\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
# Train and time
start_tf = time.time()
tf_model.fit(X_train, y_train_cat, epochs=10, verbose=0)
end_tf = time.time()
tf_time = end_tf - start_tf

# Predict and evaluate
tf_preds = np.argmax(tf_model.predict(X_test), axis=1)
tf_acc = accuracy_score(y_test, tf_preds)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

# Convert to torch tensors
X_train_torch = torch.tensor(X_train, dtype=torch.float32)
y_train_torch = torch.tensor(y_train, dtype=torch.long)
X_test_torch = torch.tensor(X_test, dtype=torch.float32)
y_test_torch = torch.tensor(y_test, dtype=torch.long)

In [7]:
# Dataset and loader
train_dataset = TensorDataset(X_train_torch, y_train_torch)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [8]:
# PyTorch model
class TorchModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(X.shape[1], 32)
        self.relu = nn.ReLU()
        self.out = nn.Linear(32, 3)
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        return self.out(x)

model = TorchModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [9]:
# Train and time
start_torch = time.time()
for epoch in range(10):
    for batch_x, batch_y in train_loader:
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
end_torch = time.time()
torch_time = end_torch - start_torch

In [10]:
# Evaluate
with torch.no_grad():
    test_outputs = model(X_test_torch)
    _, torch_preds = torch.max(test_outputs, 1)
    torch_acc = accuracy_score(y_test_torch, torch_preds)

In [11]:
import pandas as pd

results = pd.DataFrame({
    'Framework': ['TensorFlow', 'PyTorch'],
    'Train Time (s)': [round(tf_time, 4), round(torch_time, 4)],
    'Test Accuracy': [round(tf_acc, 4), round(torch_acc, 4)]
})

print(results)


    Framework  Train Time (s)  Test Accuracy
0  TensorFlow          2.1228         0.6944
1     PyTorch          0.1492         1.0000
